In [1]:
from lxml import html

from bs4 import BeautifulSoup
import time
import datetime

import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import NoSuchElementException

import json
import os

from apscheduler.schedulers.background import BackgroundScheduler
import sys

import helper

In [2]:
TIME_INTERVAL_SIGNIN_BOOKING = 180 #3 minutes
DELAY_TIME_BOOKING = 0.45 #0.4 too early

IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20

In [3]:
# NO_WINDOW = True

In [4]:
course_schedule = []

In [5]:
def check_booking_status_ready():
    str = drivers[0].find_element("xpath","//div[@class='class-info']/div[2]/div[2]").get_attribute("innerHTML")
    str = str.split("\n")[-2]
    # print(str)
    if "預定開始日期" in str:
        print("約課頁面還沒出現")
        return False
    else:
        return True
    
def check_booking_status_booked():
    str = drivers[0].find_element("xpath","//div[@class='class-info']/div[2]/div[2]").get_attribute("innerHTML")
    str = str.split("\n")[-2]
    
    if "您已預定此課" in str:
        print("您已預定此課")
        return True
    else:
        return False

In [6]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element("xpath",xpath)

In [7]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')
    
    send_keys(username, "DN20092360")
    send_keys(password, "jj1216")
#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))

    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()
    
def click_book_this_class_now(driver, wait):
    
    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預定此課')]")
    
    # click_btn = driver.find_element("xpath","//a[contains(text(), '現在就預定此課')]") 
    # try:
    #     click_btn = driver.find_element("id",'ctl00_cphContents_btnBook')
    #     click_btn.click()
    # except NoSuchElementException:
    #     print("Error: click_book_this_class_now -- cannot find the book now button")
    
    print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False


In [8]:
def scheduled_sign_in(driver, wait, class_to_be_booked):
    print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    time.sleep(1)
    sign_in(driver, wait)

    
def reserve_class(driver, wait, class_to_be_booked):
    
    print("reserve_class: ({})\n".format(class_to_be_booked['time']))
    
    book_url = class_to_be_booked['href']
    class_name = class_to_be_booked['name']
    
    driver.get(book_url)

    click_book_this_class_now(driver, wait)

In [9]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [10]:
def create_new_driver():
    #start web driver

    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=r'/usr/bin/chromedriver')

    #setup option for chrome profile
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    

    return driver, wait

In [11]:
# load schedule from file
with open("./class_schedules/06 十一月 - 12 十一月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)

In [12]:
# setup classes to be booked
classes_to_be_booked = []

##---------------------------------------------------------------
Day_string = "16"

c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Vin', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('pm', c)

classes_to_be_booked.extend(c)

c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Gen', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('pm', c)

classes_to_be_booked.extend(c)
##---------------------------------------------------------------

classes_to_be_booked

[{'room': '敦南 6 教室',
  'date': '12 11月 周日',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242726',
  'name': 'Vinyasa Yoga 流暢瑜珈',
  'teacher': 'Una',
  'time': '08:30 - 09:30 上午'},
 {'room': '敦南 6 教室',
  'date': '12 11月 周日',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242690',
  'name': 'Gentle Yoga 和緩瑜珈',
  'teacher': 'Una',
  'time': '09:45 - 10:45 上午'}]

[{'room': '敦南 6 教室',
  'date': '12 11月 周日',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242726',
  'name': 'Vinyasa Yoga 流暢瑜珈',
  'teacher': 'Una',
  'time': '08:30 - 09:30 上午'},
 {'room': '敦南 6 教室',
  'date': '12 11月 周日',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242690',
  'name': 'Gentle Yoga 和緩瑜珈',
  'teacher': 'Una',
  'time': '09:45 - 10:45 上午'}]

In [13]:
# retrieve class date and time, convert it in python's datetime structure then schedule a process to book!

drivers = []

scheduler = BackgroundScheduler()

for idx,class_to_be_booked in enumerate(classes_to_be_booked):

    #retrieve class date and time, convert it in python's datetime structure
    
    date_comp = class_to_be_booked['date'].split(" ")
    date_comp[1] = date_comp[1].replace("月","")
    date_comp.pop()
    month = int(date_comp[1])
    day = int(date_comp[0])
    # month,day

    time_comp = class_to_be_booked['time'].split(" ")
    hour = time_comp[0].split(":")[0]
    end_hour = time_comp[2].split(":")[0]
    end_hour = int(end_hour)
    minute = time_comp[0].split(":")[1]
    hour = int(hour)
    minute = int(minute)

    if time_comp[3] == "下午":
        print(time_comp)
        # deal with classes that start in the morning but end in the afternoon
        if end_hour == 12 or hour == 12:
            pass
        else:
            hour = hour+12

    # hour,minute

    from datetime import date
    today = date.today()
    year = int(today.strftime("%Y"))
    class_datetime = datetime.datetime(year, month, day, hour, minute, 0)
    book_datetime = class_datetime - datetime.timedelta(seconds=72*60*60)
    print("book_datetime= {}".format(book_datetime))

    # setup web driver and scheduled process for each class

    driver, wait = create_new_driver()
    drivers.append(driver)

    now = datetime.datetime.now()

    if (book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)) < now:
        print("sign in and book immediately")
        
        signin_datetime = now + datetime.timedelta(seconds=4+idx)
        book_datetime = now + datetime.timedelta(seconds=10+idx)
        if book_datetime < now:
            print("too late for booking")
            continue

    else:
        signin_datetime = book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)
    
    book_datetime = book_datetime + datetime.timedelta(seconds=DELAY_TIME_BOOKING)
    
    print("idx: ", idx, "scheduled signin datetime: ", signin_datetime)
    print("idx: ", idx, "scheduled book datetime: ", book_datetime)
    
    scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime, args=[driver, wait, class_to_be_booked])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, class_to_be_booked])

scheduler.start()
drivers

book_datetime= 2023-11-09 08:30:00
book_datetime= 2023-11-09 08:30:00
idx:  0 scheduled signin datetime:  2023-11-09 08:27:00
idx:  0 scheduled book datetime:  2023-11-09 08:30:00.400000
book_datetime= 2023-11-09 09:45:00
idx:  0 scheduled signin datetime:  2023-11-09 08:27:00
idx:  0 scheduled book datetime:  2023-11-09 08:30:00.400000
book_datetime= 2023-11-09 09:45:00
idx:  1 scheduled signin datetime:  2023-11-09 09:42:00
idx:  1 scheduled book datetime:  2023-11-09 09:45:00.400000
idx:  1 scheduled signin datetime:  2023-11-09 09:42:00
idx:  1 scheduled book datetime:  2023-11-09 09:45:00.400000


[<selenium.webdriver.chrome.webdriver.WebDriver (session="d16c5fc6f10f967806e7fd831ca6b5cd")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="c9c6df09a64261b9cd03ac282f83e330")>]

[<selenium.webdriver.chrome.webdriver.WebDriver (session="d16c5fc6f10f967806e7fd831ca6b5cd")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="c9c6df09a64261b9cd03ac282f83e330")>]

scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="d16c5fc6f10f967806e7fd831ca6b5cd")>,<selenium.webdriver.support.wait.WebDriverWait (session="d16c5fc6f10f967806e7fd831ca6b5cd")>)
scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="d16c5fc6f10f967806e7fd831ca6b5cd")>,<selenium.webdriver.support.wait.WebDriverWait (session="d16c5fc6f10f967806e7fd831ca6b5cd")>)
reserve_class: (08:30 - 09:30 上午)

reserve_class: (08:30 - 09:30 上午)



Job "reserve_class (trigger: date[2023-11-09 08:30:00 CST], next run at: 2023-11-09 08:30:00 CST)" raised an exception
Traceback (most recent call last):
  File "/home/huohsien/.anaconda3/envs/web3.8/lib/python3.8/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/tmp/ipykernel_1249541/3002691882.py", line 17, in reserve_class
    click_book_this_class_now(driver, wait)
  File "/tmp/ipykernel_1249541/286820193.py", line 22, in click_book_this_class_now
    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預定此課')]")
  File "/tmp/ipykernel_1249541/1835086340.py", line 10, in sync_get_element_by_xpath
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
  File "/home/huohsien/.anaconda3/envs/web3.8/lib/python3.8/site-packages/selenium/webdriver/support/wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutExcepti

scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="c9c6df09a64261b9cd03ac282f83e330")>,<selenium.webdriver.support.wait.WebDriverWait (session="c9c6df09a64261b9cd03ac282f83e330")>)
scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="c9c6df09a64261b9cd03ac282f83e330")>,<selenium.webdriver.support.wait.WebDriverWait (session="c9c6df09a64261b9cd03ac282f83e330")>)
reserve_class: (09:45 - 10:45 上午)

reserve_class: (09:45 - 10:45 上午)



Job "reserve_class (trigger: date[2023-11-09 09:45:00 CST], next run at: 2023-11-09 09:45:00 CST)" raised an exception
Traceback (most recent call last):
  File "/home/huohsien/.anaconda3/envs/web3.8/lib/python3.8/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/tmp/ipykernel_1249541/3002691882.py", line 17, in reserve_class
    click_book_this_class_now(driver, wait)
  File "/tmp/ipykernel_1249541/286820193.py", line 22, in click_book_this_class_now
    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預定此課')]")
  File "/tmp/ipykernel_1249541/1835086340.py", line 10, in sync_get_element_by_xpath
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
  File "/home/huohsien/.anaconda3/envs/web3.8/lib/python3.8/site-packages/selenium/webdriver/support/wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutExcepti